# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv


# Dataset Loading

In [2]:
train = pd.read_csv(r'D:\Project Phase II\Dataset\finaltrain.csv',encoding='cp1252')
test = pd.read_csv(r'D:\Project Phase II\Dataset\finaltest.csv',encoding='cp1252') #test_set1 
autest = pd.read_csv(r'D:\Project Phase II\Dataset\UNSW_NB15_testing-set.csv',encoding='cp1252') #test_set2

C:\Users\sathi\AppData\Local\Temp\ipykernel_17236\3389183221.py:1: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(r'D:\Project Phase II\Dataset\finaltrain.csv',encoding='cp1252')
C:\Users\sathi\AppData\Local\Temp\ipykernel_17236\3389183221.py:2: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(r'D:\Project Phase II\Dataset\finaltest.csv',encoding='cp1252') #test_set1


# Data Preprocessing

## Preprocess user-made train-test datasets

In [3]:
todrop = ['sloss', 'dloss','stcpb', 'dtcpb', 'trans_depth', 'Stime', 'Ltime','ct_flw_http_mthd', 
        'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
        'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat']
        
reducedTrain = train.drop(todrop, axis = 1)
reducedTest = test.drop(todrop, axis = 1)
reducedTrain = reducedTrain.drop_duplicates()
reducedTest = reducedTest.drop_duplicates()

trainAttributes = reducedTrain.drop(['srcip','sport','dstip','dsport','Label'], axis = 1)
trainLabel = reducedTrain['Label']
testAttributes = reducedTest.drop(['srcip','sport','dstip','dsport','Label'], axis = 1)
testLabel = reducedTest['Label']

train = reducedTrain
test = reducedTest

train = train.drop_duplicates(['srcip','sport', 'dstip', 'dsport'], keep = 'last')
test = test.drop_duplicates(['srcip','sport', 'dstip', 'dsport'], keep = 'last')

In [7]:
print("Train Set:\n",train.Label.value_counts())
print("Test Set:\n", test.Label.value_counts())

Train Set:
 Label
0    39990
1    39821
Name: count, dtype: int64
Test Set:
 Label
1    17217
0    17201
Name: count, dtype: int64


## Preprocess Author's testing dataset

In [4]:
autestdrop = ['ï»¿id', 'rate', 'sloss', 'dloss','stcpb', 'dtcpb','trans_depth','ct_srv_src','ct_dst_ltm',
        'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
        'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
        'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat']

autest = autest.drop(autestdrop, axis = 1)
#Change column names of autest
autest.rename(columns={'dinpkt':'Dintpkt','djit':'Djit','dload':'Dload','dpkts':'Dpkts','label':'Label','sinpkt':'Sintpkt','sjit':'Sjit', 'sload':'Sload', 'spkts':'Spkts','dmean':'dmeansz','response_body_len':'res_bdy_len', 'smean':'smeansz'}, inplace=True)
autest = autest[(autest['proto']=='tcp') | (autest['proto']=='udp') | (autest['proto']=='ospf')]
autest = autest[(autest['service']=='ssh') | (autest['service']=='ftp-data ')  | (autest['service']=='ftp') | (autest['service']=='-') | (autest['service']=='dns') | (autest['service']=='smtp') | (autest['service']=='http') | (autest['service']=='radius') | (autest['service']=='pop3') ]
autest = autest[(autest['state'] == 'CON') | (autest['state'] == 'RST') | (autest['state'] == 'FIN') | (autest['state'] == 'ACC') | (autest['state'] == 'REQ') | (autest['state'] == 'INT')]

autest['proto'].replace("tcp", 0, inplace = True)
autest['proto'].replace("udp", 1, inplace = True)
autest['proto'].replace("ospf", 2, inplace = True)

autest['service'].replace("ssh", 0, inplace = True)
autest['service'].replace("ftp-data", 1, inplace = True)
autest['service'].replace("ftp", 2, inplace = True)
autest['service'].replace("-", 3, inplace = True)
autest['service'].replace("dns", 4, inplace = True)
autest['service'].replace("smtp", 5, inplace = True)
autest['service'].replace("http", 6, inplace = True)
autest['service'].replace("radius", 7, inplace = True)
autest['service'].replace("pop3", 8, inplace = True)

autest['state'].replace("CON", 0, inplace = True)
autest['state'].replace("RST", 1, inplace = True)
autest['state'].replace("FIN", 2, inplace = True)
autest['state'].replace("ACC", 3, inplace = True)
autest['state'].replace("REQ", 4, inplace = True)
autest['state'].replace("INT", 5, inplace = True)

for column in autest.columns:
        if column != 'Label':
                col_mean = sum(autest[column]) / len(autest[column])
                col_std = (sum((x - col_mean) ** 2 for x in autest[column]) / len(autest[column])) ** 0.5
                autest[column] = [(x - col_mean) / col_std for x in autest[column]]


autest = autest.drop_duplicates()
autestAttributes = autest.drop(['Label'], axis = 1)
autestAttributes = autestAttributes[trainAttributes.columns]
autestLabel = autest['Label']

In [59]:
print("Author's Testing Set:\n", autest.Label.value_counts())

Author's Testing Set:
 Label
0    29673
1    14973
Name: count, dtype: int64


# Input Transformation

## Training

In [5]:
nodes = set()
trainUnique = train
for i in range(len(train)):
    src = str(train['srcip'].iloc[i])+':'+str(train['sport'].iloc[i])
    dst = str(train['dstip'].iloc[i])+':'+str(train['dsport'].iloc[i])
    nodes.add(src)
    nodes.add(dst)


#Find unique (srcip,sport,dstip,dsport) tuples in train
train_tuples = set()
for i in range(len(train)):
    train_tuples.add((train['srcip'].iloc[i], train['sport'].iloc[i], train['dstip'].iloc[i], train['dsport'].iloc[i]))

In [6]:
xLookUp = {}
x = []
count = 0
edge_index = []
edge_attr = []
for i in range(len(train)):
    src = str(train['srcip'].iloc[i])+':'+str(train['sport'].iloc[i])
    dst = str(train['dstip'].iloc[i])+':'+str(train['dsport'].iloc[i])
    if src not in xLookUp:
        xLookUp[src] = count
        x.append([int(train['sport'].iloc[i])])
        count += 1
    if dst not in xLookUp:
        xLookUp[dst] = count
        x.append([int(train['dsport'].iloc[i])])
        count += 1
    edge_index.append([xLookUp[src], xLookUp[dst]])
    edge_attr.append(list(train.iloc[i].drop(['srcip', 'sport', 'dstip', 'dsport','Label']).values))


# Convert x to tensor
x_tensor = torch.tensor(x, dtype=torch.float)

# Convert edge_index to tensor
edge_index_tensor = torch.tensor(edge_index, dtype=torch.long).t()  # Transpose for correct format

# Convert edge_attr to tensor
edge_attr_tensor = torch.tensor(edge_attr, dtype=torch.float)

print(x_tensor.shape, edge_index_tensor.shape, edge_attr_tensor.shape)

torch.Size([94760, 1]) torch.Size([2, 79811]) torch.Size([79811, 25])


## Testing

In [7]:
testnodes = set()
testUnique = test
for i in range(len(test)):
    src = str(test['srcip'].iloc[i])+':'+str(test['sport'].iloc[i])
    dst = str(test['dstip'].iloc[i])+':'+str(test['dsport'].iloc[i])
    testnodes.add(src)
    testnodes.add(dst)


#Find unique (srcip,sport,dstip,dsport) tuples in train
test_tuples = set()
for i in range(len(test)):
    test_tuples.add((test['srcip'].iloc[i], test['sport'].iloc[i], test['dstip'].iloc[i], test['dsport'].iloc[i]))

In [8]:
xtestLookUp = {}
xtest = []
count = 0
edge_index_test = []
edge_attr_test = []
for i in range(len(test)):
    src = str(test['srcip'].iloc[i])+':'+str(test['sport'].iloc[i])
    dst = str(test['dstip'].iloc[i])+':'+str(test['dsport'].iloc[i])
    if src not in xtestLookUp:
        xtestLookUp[src] = count
        xtest.append([int(test['sport'].iloc[i])])
        count += 1
    if dst not in xtestLookUp:
        xtestLookUp[dst] = count
        xtest.append([int(test['dsport'].iloc[i])])
        count += 1
    edge_index_test.append([xtestLookUp[src], xtestLookUp[dst]])
    edge_attr_test.append(list(test.iloc[i].drop(['srcip', 'sport', 'dstip', 'dsport','Label']).values))


# Convert x to tensor
x_test_tensor = torch.tensor(xtest, dtype=torch.float)

# Convert edge_index to tensor
edge_index_test_tensor = torch.tensor(edge_index_test, dtype=torch.long).t()  # Transpose for correct format

# Convert edge_attr to tensor
edge_attr_test_tensor = torch.tensor(edge_attr_test, dtype=torch.float)

print(x_test_tensor.shape, edge_index_test_tensor.shape, edge_attr_test_tensor.shape)

torch.Size([42412, 1]) torch.Size([2, 34418]) torch.Size([34418, 25])


## Author's Testing Dataset

Since srcip, sport, dstip, dsport information is not available, we proceed with having each node feature value as 0, and assume each edge as an unique connection between two imaginary nodes

In [9]:
n_autestnodes = int(len(autest) ** 0.5)
n_autest_tuples = n_autestnodes ** 2

xautest = [[0] for j in range(n_autest_tuples)]
edge_index_autest = []
edge_attr_autest = []
auEdgeLabel = []
count = 0
for i in range(n_autestnodes):
    for j in range(n_autestnodes):
        if(i == count):
            count += 1
            continue
        edge_index_autest.append([i, count])
        edge_attr_autest.append(list(autest.iloc[count].drop(['Label']).values))
        count += 1
        auEdgeLabel.append(autest.iloc[count].Label)
        

# Convert x to tensor
x_autest_tensor = torch.tensor(xautest, dtype=torch.float)

# Convert edge_index to tensor
edge_index_autest_tensor = torch.tensor(edge_index_autest, dtype=torch.long).t()  # Transpose for correct format

# Convert edge_attr to tensor
edge_attr_autest_tensor = torch.tensor(edge_attr_autest, dtype=torch.float)

print(x_autest_tensor.shape, edge_index_autest_tensor.shape, edge_attr_autest_tensor.shape)

torch.Size([44521, 1]) torch.Size([2, 44520]) torch.Size([44520, 25])


# GAT Model

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GATClassifier(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features, hidden_channels, num_heads):
        super(GATClassifier, self).__init__()
        self.conv1 = GATConv(num_node_features, hidden_channels, heads=num_heads, add_self_loops=False)
        self.conv2 = GATConv(hidden_channels * num_heads, hidden_channels, heads=num_heads, add_self_loops=False)
        # self.edge_classifier = nn.Linear(hidden_channels * num_heads + num_edge_features, 1)  # Output size changed to 1
        self.edge_classifier = nn.Linear(hidden_channels * 2 * num_heads + num_edge_features, 1)  # Output size changed to 1

    def forward(self, x, edge_index, edge_attr):
        # Apply graph attentional layers
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)

        # Concatenate node features with edge features
        num_edges = edge_index.size(1)
        node_x = x[edge_index[0]]
        node_x_other = x[edge_index[1]]
        edge_attr_padded = torch.cat([edge_attr, torch.zeros(num_edges - edge_attr.size(0), edge_attr.size(1))], dim=0)
        edge_x = torch.cat([node_x, node_x_other, edge_attr_padded], dim=1)
        print("edge_x.shape:", edge_x.shape)
        print("self.edge_classifier.weight.shape:",self.edge_classifier.weight.shape)
        # Classify edges into binary class using BCELogit
        edge_scores = self.edge_classifier(edge_x)
        return edge_scores


Training

In [33]:
# Example usage
num_nodes = len(nodes)
num_edges = len(train_tuples)
num_node_features = 1
num_edge_features = 25
hidden_channels = 16 
num_heads = 4

# Instantiate GAT classifier
model = GATClassifier(num_node_features, num_edge_features, hidden_channels, num_heads)

# Forward pass
edge_scores_train = model(x_tensor, edge_index_tensor, edge_attr_tensor)

edge_x.shape: torch.Size([79811, 153])
self.edge_classifier.weight.shape: torch.Size([1, 153])


Testing

In [34]:
# Example usage
num_nodes_test = len(testnodes)
num_edges_test = len(test_tuples)
num_node_features = 1
num_edge_features = 25
hidden_channels = 16
num_heads = 4

# Forward pass
edge_scores_test = model(x_test_tensor, edge_index_test_tensor, edge_attr_test_tensor)

edge_x.shape: torch.Size([34418, 153])
self.edge_classifier.weight.shape: torch.Size([1, 153])


Author's testing dataset

In [35]:
# Example usage
num_nodes_test = n_autestnodes
num_edges_test = n_autest_tuples
num_node_features = 1
num_edge_features = 25
hidden_channels = 16
num_heads = 4

# Forward pass
edge_scores_autest = model(x_autest_tensor, edge_index_autest_tensor, edge_attr_autest_tensor)

edge_x.shape: torch.Size([44520, 153])
self.edge_classifier.weight.shape: torch.Size([1, 153])


# KNN

In [36]:
import pandas as pd
# Convert edge_scores tensor to a NumPy array
edge_scores_train_array = edge_scores_train.detach().numpy()
edge_scores_test_array = edge_scores_test.detach().numpy()
# Create a DataFrame from the NumPy array
edge_scores_train_df = pd.DataFrame(edge_scores_train_array, columns=['Edge_Scores'])
edge_scores_test_df = pd.DataFrame(edge_scores_test_array, columns=['Edge_Scores'])
trainLabel = train['Label']
testLabel = test['Label']

In [37]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

X_train = edge_scores_train_df
X_test = edge_scores_test_df
y_train = trainLabel
y_test = testLabel

# Initialize the KNN classifier
k = 25  # Number of neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the classifier
knn_classifier.fit(X_train, y_train)

# Predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Calculate accuracy
report = classification_report(y_test, y_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

           0       0.98      0.96      0.97     17201
           1       0.96      0.98      0.97     17217

    accuracy                           0.97     34418
   macro avg       0.97      0.97      0.97     34418
weighted avg       0.97      0.97      0.97     34418



Author's test dataset

In [38]:
import pandas as pd
# Convert edge_scores tensor to a NumPy array
edge_scores_autest_array = edge_scores_autest.detach().numpy()
# Create a DataFrame from the NumPy array
testLabeldf = pd.DataFrame(auEdgeLabel, columns=['Label'])
edge_scores_autest_df = pd.DataFrame(edge_scores_autest_array, columns=['Edge_Scores'])
trainLabel = train['Label']
autestLabel = testLabeldf['Label']
auX_test = edge_scores_autest_df
auy_test = autestLabel 

auy_pred = knn_classifier.predict(edge_scores_autest_df)

# Calculate accuracy
report = classification_report(autestLabel, auy_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

         0.0       0.62      0.75      0.68     29547
         1.0       0.16      0.10      0.12     14973

    accuracy                           0.53     44520
   macro avg       0.39      0.42      0.40     44520
weighted avg       0.47      0.53      0.49     44520



# Lazy Predict Scores

In [39]:
# List of classifiers to include
import sklearn
import xgboost
import lightgbm
import sklearn.discriminant_analysis
from lazypredict.Supervised import LazyClassifier


classifiers = [
    ('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
    ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
    ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
    ('ExtraTreeClassifier', sklearn.ensemble._forest.ExtraTreeClassifier),
    ('ExtraTreesClassifier', sklearn.ensemble._forest.ExtraTreesClassifier),
    ('KNeighborsClassifier',  sklearn.neighbors._classification.KNeighborsClassifier),
    ('NuSVC', sklearn.svm._classes.NuSVC),
    ('RandomForestClassifier', sklearn.ensemble._forest.RandomForestClassifier),
    ('SVC', sklearn.svm._classes.SVC),
    ('XGBClassifier', xgboost.sklearn.XGBClassifier),
    ('LGBMClassifier', lightgbm.sklearn.LGBMClassifier)]
clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None,classifiers= classifiers, predictions=True)

models,predictions = clf.fit(X_train, X_test, y_train, y_test)


'tuple' object has no attribute '__name__'
Invalid Classifier(s)


  9%|▉         | 1/11 [00:02<00:27,  2.78s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.9687082340635714, 'Balanced Accuracy': 0.9687030542403166, 'ROC AUC': 0.9687030542403167, 'F1 Score': 0.9687041967837421, 'Time taken': 2.7778916358947754}


 18%|█▊        | 2/11 [00:06<00:28,  3.13s/it]

{'Model': 'BaggingClassifier', 'Accuracy': 0.9520309140565983, 'Balanced Accuracy': 0.9520309442102031, 'ROC AUC': 0.952030944210203, 'F1 Score': 0.9520309156358626, 'Time taken': 3.3793466091156006}


...

{'Model': 'SVC', 'Accuracy': 0.9455808007437968, 'Balanced Accuracy': 0.9455919319996959, 'ROC AUC': 0.9455919319996959, 'F1 Score': 0.945550123003457, 'Time taken': 613.1930994987488}


 91%|█████████ | 10/11 [25:03<03:38, 218.99s/it]

{'Model': 'XGBClassifier', 'Accuracy': 0.9686501249346272, 'Balanced Accuracy': 0.9686437565104128, 'ROC AUC': 0.9686437565104128, 'F1 Score': 0.968644053359387, 'Time taken': 0.7819991111755371}
[LightGBM] [Info] Number of positive: 39821, number of negative: 39990
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002951 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 79811, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498941 -> initscore=-0.004235
[LightGBM] [Info] Start training from score -0.004235


100%|██████████| 11/11 [25:04<00:00, 136.78s/it]

{'Model': 'LGBMClassifier', 'Accuracy': 0.9687372886280434, 'Balanced Accuracy': 0.9687316901037897, 'ROC AUC': 0.9687316901037897, 'F1 Score': 0.9687325897738376, 'Time taken': 1.0422351360321045}


In [40]:
from sklearn.metrics import roc_auc_score
for m in predictions.columns:
    print(m)
    print(classification_report(y_test, predictions[m]))
    auc_scores = roc_auc_score(y_test, predictions[m], multi_class='ovr')  # Or multi_class='ovo' for one-vs-one
    print("AUC Scores for each class:", auc_scores)
    print('-----------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------')

AdaBoostClassifier
              precision    recall  f1-score   support

           0       0.98      0.96      0.97     17201
           1       0.96      0.98      0.97     17217

    accuracy                           0.97     34418
   macro avg       0.97      0.97      0.97     34418
weighted avg       0.97      0.97      0.97     34418

AUC Scores for each class: 0.9687030542403167
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
BaggingClassifier
              precision    recall  f1-score   support

           0       0.95      0.95      0.95     17201
           1       0.95      0.95      0.95     17217

    accuracy                           0.95     34418
   macro avg       0.95      0.95      0.95     34418
weighted avg       0.95      0.95      0.95     34418

AUC Scores for each class: 0.952030944210203
--------------------------------------------------

Lazy Predict for author's testing dataset

In [41]:
# List of classifiers to include
import sklearn
import xgboost
import lightgbm
import sklearn.discriminant_analysis
from lazypredict.Supervised import LazyClassifier

classifiers = [
    ('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
    ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
    ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
    ('ExtraTreeClassifier', sklearn.ensemble._forest.ExtraTreeClassifier),
    ('ExtraTreesClassifier', sklearn.ensemble._forest.ExtraTreesClassifier),
    ('KNeighborsClassifier',  sklearn.neighbors._classification.KNeighborsClassifier),
    ('NuSVC', sklearn.svm._classes.NuSVC),
    ('RandomForestClassifier', sklearn.ensemble._forest.RandomForestClassifier),
    ('SVC', sklearn.svm._classes.SVC),
    ('XGBClassifier', xgboost.sklearn.XGBClassifier),
    ('LGBMClassifier', lightgbm.sklearn.LGBMClassifier)]
clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None,classifiers= classifiers, predictions=True)

aumodels,aupredictions = clf.fit(X_train, auX_test, y_train, auy_test)

'tuple' object has no attribute '__name__'
Invalid Classifier(s)


  9%|▉         | 1/11 [00:03<00:37,  3.74s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.530256064690027, 'Balanced Accuracy': 0.41985711689237737, 'ROC AUC': 0.4198571168923774, 'F1 Score': 0.4878497165706889, 'Time taken': 3.741649866104126}


 18%|█▊        | 2/11 [00:07<00:34,  3.83s/it]

{'Model': 'BaggingClassifier', 'Accuracy': 0.546900269541779, 'Balanced Accuracy': 0.4395449833865848, 'ROC AUC': 0.4395449833865848, 'F1 Score': 0.5071680314421203, 'Time taken': 3.894402265548706}


...

{'Model': 'SVC', 'Accuracy': 0.5778301886792453, 'Balanced Accuracy': 0.4490768766390258, 'ROC AUC': 0.44907687663902585, 'F1 Score': 0.5092230800353242, 'Time taken': 764.0786874294281}


 91%|█████████ | 10/11 [34:04<05:01, 301.58s/it]

{'Model': 'XGBClassifier', 'Accuracy': 0.5197439353099731, 'Balanced Accuracy': 0.41773541106519574, 'ROC AUC': 0.41773541106519574, 'F1 Score': 0.48713326807359086, 'Time taken': 0.9190926551818848}
[LightGBM] [Info] Number of positive: 39821, number of negative: 39990
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001510 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 79811, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498941 -> initscore=-0.004235
[LightGBM] [Info] Start training from score -0.004235


100%|██████████| 11/11 [34:05<00:00, 185.93s/it]

{'Model': 'LGBMClassifier', 'Accuracy': 0.5310646900269542, 'Balanced Accuracy': 0.42100986703809096, 'ROC AUC': 0.42100986703809096, 'F1 Score': 0.4890194857752686, 'Time taken': 1.0510048866271973}


In [42]:
from sklearn.metrics import roc_auc_score
for model in aupredictions.columns:
    print(model)
    print(classification_report(auy_test, aupredictions[model]))
    auc_scores = roc_auc_score(auy_test, aupredictions[model], multi_class='ovr')  # Or multi_class='ovo' for one-vs-one
    print("AUC Scores for each class:", auc_scores)
    print('-----------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------')

AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.62      0.76      0.68     29547
         1.0       0.15      0.08      0.11     14973

    accuracy                           0.53     44520
   macro avg       0.38      0.42      0.39     44520
weighted avg       0.46      0.53      0.49     44520

AUC Scores for each class: 0.4198571168923774
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
BaggingClassifier
              precision    recall  f1-score   support

         0.0       0.63      0.77      0.69     29547
         1.0       0.20      0.11      0.14     14973

    accuracy                           0.55     44520
   macro avg       0.41      0.44      0.42     44520
weighted avg       0.48      0.55      0.51     44520

AUC Scores for each class: 0.4395449833865848
-------------------------------------------------